<a href="https://colab.research.google.com/github/Shehbaz-Niazi/Agentic_AI-PROJECTS/blob/main/RAG_Advance_With_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q -U google-generativeai
!pip install -Uq langchain-chroma
!pip install -Uq langchain-google-genai


In [37]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))


In [ ]:
list(genai.list_models())

In [ ]:
result = genai.embed_content(
    model="models/text-embedding-004",
    content="Pakistan zinda bad we love our country",
    task_type="retrieval_document",
    title="Embedding of single string",
)

result["embedding"]

In [9]:
len(result['embedding'])

768

In [ ]:
result = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would a woodchuck chuck?",
        "How does the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of list of strings",
)

# A list of inputs > A list of vectors output
for v in result["embedding"]:
    print(str(v)[:50], "... TRIMMED ...", len(v))

# **Building Vector Stores & Retreival using Chroma DB and Langchain**

In [12]:
!pip install -Uq langchain-chroma

In [13]:
import getpass
import os

In [14]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",
                                          google_api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
embeddings.embed_query("What's our Q1 revenue?")

In [17]:
from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings)

In [ ]:
list(dir(vectorstore))

In [19]:
vectorstore

In [ ]:
vectorstore.similarity_search("cat")

In [ ]:
await vectorstore.asimilarity_search("cat")

In [ ]:
vectorstore.similarity_search_with_score("cat")

In [ ]:
embedding = embeddings.embed_query("cat")# convert cat into vector

vectorstore.similarity_search_by_vector(embedding)

In [29]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["shark"])

[[Document(id='ae008101-67f9-49f2-bf45-45dac7d018c9', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = userdata.get('GOOGLE_API_KEY')
)

In [ ]:
# llm.invoke("tell about cats?")

In [31]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

In [32]:
prompt = ChatPromptTemplate.from_messages([("human", message)])

# **RAG**

In [33]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [35]:
response = rag_chain.invoke("tell about cat?")

print(response.content)

Cats are independent pets that often enjoy their own space.
